In [ ]:
#상어
#우선순위에대한 것이 필요해보임 
import sys
from collections import defaultdict
scale = 2
N = int(sys.stdin.readline())
MAP = []
impossible = True
for i in range(N):
    temp = list(map(int,sys.stdin.readline().split()))
    MAP.append(temp)
    for index,j in enumerate(temp):
        if j < scale:
            impossible = False
        if j == 9:
            x,y = len(MAP)-1,index
            MAP[-1][index] = 0
def bfs(nx,ny):
    global ddx,ddy
    global N
    global scale
    global count
    q = [(nx,ny,0)]
    visited = defaultdict(bool)
    while q:
        x,y,t = q.pop(0)
        visited[(x,y)] = True
        for dx, dy in zip(ddx,ddy):
            nx,ny = x+dx, y+dy
            if 0<=nx<N and 0<=ny< N:
                if 0<MAP[nx][ny]<=scale and not visited[(nx,ny)]:
                    if MAP[nx][ny] == scale:
                        q.append((nx,ny,t+1))
                    else:
                        count+=1
                        MAP[nx][ny] = 0
                        if count == scale:
                            count=0
                            scale+=1
                        return nx,ny,t+1
                elif MAP[nx][ny] == 0 and not visited[(nx,ny)]:
                    q.append((nx,ny,t+1))
            
    return -1,-1,-1

if impossible : 
    print(0) 
else:
    print(x,y)
    count = 0 #몸집이 커지는 조건을 담아야함.
    time = 0
    ddx, ddy = [-1,0,0,1], [0,-1,1,0]
    while True:
        nx,ny,t = bfs(x,y)
        if nx == -1:
            break
        else:
            time+=t
            x,y = nx,ny
    print(time)

In [ ]:
import sys
from collections import defaultdict
from copy import deepcopy

ddx = [-1, -1, 0, 1, 1, 1, 0, -1]
ddy = [0, -1, -1, -1, 0, 1, 1, 1]
def rotate(direction_idx):
    if direction_idx == 7:
        return 0
    else:
        return direction_idx+1
    
def move_fish(fish_num,shark_position,direction,position,directions,MAP):
    x,y = position[fish_num]
    nx,ny = x+ddx[direction] , y+ddy[direction]
    if nx<0 or nx>=4 or ny<0 or ny>=4 :
        directions[fish_num] = rotate(direction)
        move_fish(fish_num,shark_position,rotate(direction),position,directions,MAP)
    else:
        if (nx,ny) == shark_position :
            directions[fish_num] = rotate(direction)
            move_fish(fish_num,shark_position,rotate(direction),position,directions,MAP)
        elif MAP[nx][ny] == 0:
            position[fish_num] = (nx,ny)
            MAP[nx][ny] = fish_num
            MAP[x][y] = 0
        else:
            to_swap = MAP[nx][ny]
            position[fish_num] , position[to_swap] = (nx,ny),(x,y)
            MAP[nx][ny],MAP[x][y] = fish_num,to_swap

MAP = []
sequence = [i for i in range(1,17)]
directions = defaultdict(int)
positions = defaultdict(tuple)

for i in range(4):
    temp = list(map(int,sys.stdin.readline().split()))
    temp2=[]
    for j in [0,2,4,6]:
        fish_num = temp[j]
        temp2.append(fish_num)
        positions[fish_num] = (i,j//2)
        directions[fish_num] = temp[j+1]-1
    MAP.append(temp2)

def dfs(x,y,direction,result,MAP,sequence,positions,directions):
    global results
    for i in range(1,4):
        nx,ny = x+ddx[direction],y+ddy[direction]
        if nx<0 or nx>=4 or ny>=4 or ny<0 or len(sequence)==0:
            # maybe the trip is closed
            results.append(result)
            return
        else:
            if MAP[nx][ny] == 0 :
                continue
            x,y = nx,ny
            MAP_cp = deepcopy(MAP)
            directions_cp = deepcopy(directions)
            positions_cp = deepcopy(positions)
            sequence_cp = deepcopy(sequence)
            be_ate = MAP_cp[nx][ny]
            shark_direction = directions_cp[be_ate]
            MAP_cp[nx][ny] = 0
            sequence_cp.remove(be_ate)
            positions_cp.pop(be_ate)
            directions_cp.pop(be_ate)
            for j in sequence_cp:
                move_fish(j,(x,y),directions_cp[j],positions_cp,directions_cp,MAP_cp)
            dfs(nx,ny,shark_direction,result+be_ate,MAP_cp,sequence_cp,positions_cp,directions_cp)

results = []
first = MAP[0][0]
sequence.remove(MAP[0][0])
MAP[0][0] = 0
for j in sequence:
    move_fish(j,(0,0),directions[j],positions,directions,MAP)

dfs(0,0,directions[first],first,MAP,sequence,positions,directions)

print(max(results))
#첫 테케만 통과 ;; 

In [ ]:
import sys
from collections import defaultdict
ddx,ddy = [-1,1,0,0],[0,0,-1,1]
def dp(index): 
    index-=1
    return ddx[index],ddy[index]
N,M,k = map(int,sys.stdin.readline().split())
MAP = []

temp_direction = defaultdict(int) # 각 상어별 현재 방향 저장
directions = defaultdict(list) # 상어별 선호하는 방향 순서 저장 
smells = defaultdict(list) # 각 상어별 냄새를 남겨둔 포지션들, 큐로 저장할 것 
position = defaultdict(tuple) # 각 상어의 현재 위치 
for i in range(N): 
    temp = list(map(int,sys.stdin.readline().split()))
    MAP.append(temp)
    for j in range(len(temp)):
        if temp[j] != 0 :
            smells[temp[j]].append((i,j))
            position[temp[j]] = (i,j)
temp = list(map(int,sys.stdin.readline().split()))
for i in range(len(temp)):
    temp_direction[i+1] = temp[i]
for j in range(4*M):
    directions[j//4+1].append(list(map(int,sys.stdin.readline().split())))
time = 0
sharks = [ i for i in range(1,M+1)]
to_append = []
sharing = defaultdict(list)
for i in sharks:
    x,y = position[i]
    sharing[(x,y)].append(i) # 현재 공간(업데이트 전)에 누가 있는지 저장할 테이블 
while time<1000:
    for shark in sharks: #상어마다 
        zero = False
        for direction in directions[shark][temp_direction[shark]-1]:
            dx,dy = dp(direction) #갈 방향 
            x,y = position[shark] #현재 위치 
            nx,ny = x+dx,y+dy #갈 위치 
            if 0<=nx<N and 0<=ny<N: #인덱스가 유효하다면 
                if MAP[nx][ny] == 0:  #비어있다면 
                    if len(sharing[(nx,ny)]) != 0: #만약 거기에 누가 먼저 와있다면 
                        sharks.remove(i)
                    else: #아무도 없다면 
                        temp_direction[shark] = direction #방향 업데이트 
                        sharing[(nx,ny)].append(shark) #맵 업뎃 전에 누가 그 공간에 있는지 
                        sharing[(x,y)].remove(shark) #전에 있던 자리를 비워줌 
                        zero=True
                        break
        if not zero: #아직 찾지 못했다면 
            for direction in directions[shark][temp_direction[shark]-1]:
                dx,dy = dp(direction)
                nx,ny = position[shark]
                if 0<=nx<N and 0<=ny<N:
                    if MAP[nx][ny] == shark: 
                        temp_direction[shark] = direction
                        break
